# Cluster BTM Using KMeans

### Notebook automatically generated from your model

Model KMeans (k=4) (4), trained on 2022-09-24 19:42:08.

#### Generated on 2022-09-24 23:43:05.098051

Clustering
This notebook will reproduce the steps for clustering the dataset BTM_data_prepared.

#### Warning

The goal of this notebook is to provide an easily readable and explainable code that reproduces the main steps
of training the model. It is not complete: some of the preprocessing done by the DSS visual machine learning is not
replicated in this notebook. This notebook will not give the same results and model performance as the DSS visual machine
learning model.

Let's start with importing the required libs :

In [1]:
import sys
import dataiku
import numpy as np
import pandas as pd
import sklearn as sk
import dataiku.core.pandasutils as pdu
from dataiku.doctor.preprocessing import PCA
from collections import defaultdict, Counter

And tune pandas display options:

In [2]:
pd.set_option('display.width', 3000)
pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', 200)

#### Importing base data

The first step is to get our machine learning dataset:

In [3]:
# We apply the preparation that you defined. You should not modify this.
preparation_steps = []
preparation_output_schema = {'columns': [{'name': 'Tree ID', 'type': 'string'}, {'name': 'Longitude', 'type': 'double'}, {'name': 'Latitude', 'type': 'double'}, {'name': 'Common Name', 'type': 'string'}, {'name': 'Condition Class / Infestation Pattern', 'type': 'string'}, {'name': 'List of Trees on  residential property', 'type': 'string'}, {'name': 'BTM positive trees self reported  on residential property', 'type': 'string'}, {'name': 'BTM positive Tree Species on property', 'type': 'bigint'}, {'name': 'Dbh 1. inches', 'type': 'bigint'}, {'name': 'Dbh 2', 'type': 'bigint'}, {'name': 'Dbh 3', 'type': 'bigint'}, {'name': 'Dbh 4', 'type': 'bigint'}, {'name': 'Dbh 5', 'type': 'bigint'}, {'name': 'Dbh 6', 'type': 'bigint'}, {'name': 'Number Observed BTM Nests', 'type': 'bigint'}, {'name': 'Entry Source', 'type': 'string'}, {'name': 'BTM (Y/N)', 'type': 'boolean'}, {'name': 'Interest in Help', 'type': 'string'}, {'name': 'Tree Type', 'type': 'string'}, {'name': 'Distance to Water', 'type': 'string'}, {'name': 'Proposed Treatment Type (Insert/Injection; Manual Removal; Organic Spray)', 'type': 'string'}, {'name': 'Treatm.  Priority (1: High, 2: Med, 3: Low)', 'type': 'bigint'}, {'name': 'Geopoint', 'type': 'string'}], 'userModified': False}

ml_dataset_handle = dataiku.Dataset('BTM_data_prepared')
ml_dataset_handle.set_preparation_steps(preparation_steps, preparation_output_schema)
%time ml_dataset = ml_dataset_handle.get_dataframe(limit = 100000)

print ('Base data has %i rows and %i columns' % (ml_dataset.shape[0], ml_dataset.shape[1]))
# Five first records",
ml_dataset.head(5)

CPU times: user 23.7 ms, sys: 6.4 ms, total: 30.1 ms
Wall time: 71.4 ms
Base data has 932 rows and 23 columns


,Tree ID,Longitude,Latitude,Common Name,Condition Class / Infestation Pattern,List of Trees on residential property,BTM positive trees self reported on residential property,BTM positive Tree Species on property,Dbh 1. inches,Dbh 2,Dbh 3,Dbh 4,Dbh 5,Dbh 6,Number Observed BTM Nests,Entry Source,BTM (Y/N),Interest in Help,Tree Type,Distance to Water,Proposed Treatment Type (Insert/Injection; Manual Removal; Organic Spray),"Treatm. Priority (1: High, 2: Med, 3: Low)",Geopoint
0,130.0,-69.633445,44.551271,Crabapple,Good,NaN,NaN,1,10,0,0,0,0,0,75,Bartlett Invent Feb 2022,Y,NaN,Fruiting/Flowering,>250,Organic Spray,1,POINT(-69.633445106 44.551270843)
1,101.0,-69.629734,44.549535,Crabapple,Fair,NaN,NaN,1,21,0,0,0,0,0,50,Bartlett Invent Feb 2022,Y,NaN,Fruiting/Flowering,>250,Organic Spray,1,POINT(-69.629734075 44.549535104)
2,59.0,-69.627131,44.551959,Elm-American,Good,NaN,NaN,1,4,0,0,0,0,0,50,Bartlett Invent Feb 2022,Y,NaN,Ornamental,<25,Insert/Injection,1,POINT(-69.627130501 44.551959107)
3,105.0,-69.629361,44.549576,Elm-American,Good,NaN,NaN,1,42,0,0,0,0,0,50,Bartlett Invent Feb 2022,Y,NaN,Ornamental,>250,Insert/Injection,1,POINT(-69.629360613 44.549576086)
4,353.0,-69.662301,44.528730,Oak-Northern Red,Good,NaN,NaN,1,24,0,0,0,0,0,50,Bartlett Invent Feb 2022,Y,NaN,Ornamental,>250,Insert/Injection,1,POINT(-69.662300718 44.528730089)


#### Initial data management

The preprocessing aims at making the dataset compatible with modeling.
At the end of this step, we will have a matrix of float numbers, with no missing values.
We'll use the features and the preprocessing steps defined in Models.

Let's only keep selected features

In [4]:
ml_dataset = ml_dataset[['Condition Class / Infestation Pattern', 'Distance to Water']]

Let's first coerce categorical columns into unicode, numerical features into floats.

In [5]:
# astype('unicode') does not work as expected

def coerce_to_unicode(x):
    if sys.version_info < (3, 0):
        if isinstance(x, str):
            return unicode(x,'utf-8')
        else:
            return unicode(x)
    else:
        return str(x)


categorical_features = ['Condition Class / Infestation Pattern', 'Distance to Water']
numerical_features = []
text_features = []
from dataiku.doctor.utils import datetime_to_epoch
for feature in categorical_features:
    ml_dataset[feature] = ml_dataset[feature].apply(coerce_to_unicode)
for feature in text_features:
    ml_dataset[feature] = ml_dataset[feature].apply(coerce_to_unicode)
for feature in numerical_features:
    if ml_dataset[feature].dtype == np.dtype('M8[ns]') or (hasattr(ml_dataset[feature].dtype, 'base') and ml_dataset[feature].dtype.base == np.dtype('M8[ns]')):
        ml_dataset[feature] = datetime_to_epoch(ml_dataset[feature])
    else:
        ml_dataset[feature] = ml_dataset[feature].astype('double')

Let's copy our dataset to keep it for eventual profiling at the end.

In [6]:
# train dataset will be the one on which we will apply ml technics
train = ml_dataset.copy()

#### Features preprocessing

The first thing to do at the features level is to handle the missing values.
Let's reuse the settings defined in the model

In [7]:
drop_rows_when_missing = []
impute_when_missing = []

# Features for which we drop rows with missing values"
for feature in drop_rows_when_missing:
    train = train[train[feature].notnull()]
    
    print ('Dropped missing records in %s' % feature)

# Features for which we impute missing values"
for feature in impute_when_missing:
    if feature['impute_with'] == 'MEAN':
        v = train[feature['feature']].mean()
    elif feature['impute_with'] == 'MEDIAN':
        v = train[feature['feature']].median()
    elif feature['impute_with'] == 'CREATE_CATEGORY':
        v = 'NULL_CATEGORY'
    elif feature['impute_with'] == 'MODE':
        v = train[feature['feature']].value_counts().index[0]
    elif feature['impute_with'] == 'CONSTANT':
        v = feature['value']
    train[feature['feature']] = train[feature['feature']].fillna(v)
    
    print ('Imputed missing values in feature %s with value %s' % (feature['feature'], coerce_to_unicode(v)))

We can now handle the categorical features (still using the settings defined in Models):

Let's dummy-encode the following features.
A binary column is created for each of the 100 most frequent values.

In [8]:
LIMIT_DUMMIES = 100

categorical_to_dummy_encode = ['Condition Class / Infestation Pattern', 'Distance to Water']

# Only keep the top 100 values
def select_dummy_values(train, features):
    dummy_values = {}
    for feature in categorical_to_dummy_encode:
        values = [
            value
            for (value, _) in Counter(train[feature]).most_common(LIMIT_DUMMIES)
        ]
        dummy_values[feature] = values
    return dummy_values

DUMMY_VALUES = select_dummy_values(train, categorical_to_dummy_encode)

def dummy_encode_dataframe(df):
    for (feature, dummy_values) in DUMMY_VALUES.items():
        for dummy_value in dummy_values:
            dummy_name = u'%s_value_%s' % (feature, coerce_to_unicode(dummy_value))
            df[dummy_name] = (df[feature] == dummy_value).astype(float)
        del df[feature]
        print ('Dummy-encoded feature %s' % feature)

dummy_encode_dataframe(train)












# Rescaling is not required

Dummy-encoded feature Condition Class / Infestation Pattern
Dummy-encoded feature Distance to Water


Removing outliers

In [9]:
# Remove outliers from train set
from dataiku.doctor.preprocessing.dataframe_preprocessing import detect_outliers

outliers = detect_outliers(train, 0.9, 9, 0.01)
train = train[~outliers]

print ("%s outliers found" % (outliers.sum()))

DEBUG:dku.ml.preprocessing:Outliers detection: fitting PCA
DEBUG:dku.ml.preprocessing:Outliers detection: performing PCA
DEBUG:dku.ml.preprocessing:Outliers detection: performing cubic-root kmeans on df (932, 7)
DEBUG:dku.ml.preprocessing:Outliers detection: selecting mini-clusters
DEBUG:dku.ml.preprocessing:Outliers detection: done


3 outliers found


#### Modeling

In [10]:
from sklearn.cluster import KMeans
clustering_model = KMeans(n_clusters=4)

We can finally cluster our dataset!

In [11]:
%time clusters = clustering_model.fit_predict(train)

CPU times: user 887 ms, sys: 903 ms, total: 1.79 s
Wall time: 60.7 ms


Build up our result dataset

#### Results

Inertia

In [12]:
print (clustering_model.inertia_)

236.39540962182468


Silhouette

In [13]:
from sklearn.metrics import silhouette_score
silhouette = silhouette_score(train.values, clusters, metric='euclidean', sample_size=2000)
print ("Silhouette score :", silhouette)

Silhouette score : 0.7805756911332342


Join our original dataset with the cluster labels we found.

In [14]:
final = train.join(pd.Series(clusters, index=train.index, name='cluster'))
final['cluster'] = final['cluster'].map(lambda cluster_id: 'cluster' + str(cluster_id))

Compute the cluster sizes

In [15]:
size = pd.DataFrame({'size': final['cluster'].value_counts()})
size.head()

,size
cluster1,330
cluster3,300
cluster0,159
cluster2,140


That's it. It's now up to you to tune your preprocessing, your algo, and your analysis !
